In [1]:
import random

import numpy as np
import random
import keras
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras import models
from keras import layers

%matplotlib inline

Using TensorFlow backend.


In [2]:
# number of characters in a word.
# for instance abccba has nb_chars = 6
nb_chars = 4

# number of possible characters used during the encoding.
# for instance abcde leads to 01234 has nb_letters = 5
nb_letters = 10
#nb_letters = 26

# number of words samples to be generated for the training
nb_words = 3000

In [3]:
# total number of combinations
nb_letters**nb_chars

10000

In [4]:
def create_inputs(nb_words, nb_chars, nb_letters):
    '''Create a numpy array of nb_words rows with nb_chars columns each element
    being a random letter of nb_letters (a, b...)'''
    words = np.zeros((nb_words, nb_chars), dtype=int)
    for w in range(nb_words):
        for c in range(nb_chars):
            i = random.randint(0, nb_letters-1)
            words[w, c] = ord('a') + i
    return words


def encrypt(words, nb_words, nb_chars):
    '''Encrypt each element of a numpy array of nb_words rows with nb_chars 
    columns each item with a secret algorithm'''
    
    encrypted_words = words.copy()
    encrypted_words_probs = np.zeros((nb_words, nb_chars, nb_chars))
    
    #val_max = -1
    
    for w in range(nb_words):
        for c in range(nb_chars): # 0,1,2,3,4
            encrypted_words[w,c] = int(words[w,c]) - 49
            val = encrypted_words[w,c] - 48
            
            #if val > val_max:
            #    val_max = val
            
            # add entropy (i.e. mistakes in the encryption)
            #epsilon = random.randint(0, 100)
            #if epsilon == 5 and val != val_max:
            #val +=1
            
            #print('w:',w,', c:',c,', [wc]:', val)
            #encrypted_words_probs[w, c, val ] = 1.0
            encrypted_words[w,c] = val
    return encrypted_words

def print_output(words, nb_words, nb_chars):
    for w in range(nb_words):
        word = ''
        for c in range(nb_chars):
            word += chr(words[w,c])
        print(word)


In [5]:
nb_features = nb_chars

# This returns a tensor
inputs = layers.Input(shape=(nb_chars,), dtype='float32', name='main_input')

# a layer instance is callable on a tensor, and returns a tensor
x = layers.Dense(2048, activation='relu', name='hl_1')(inputs)
#x = layers.Dense(128, activation='relu', name='hl_1')(inputs)
#x = layers.Dense(64, activation='relu', name='hl_2')(x)

outputs = []
losses = {}
for o in range(nb_chars):
    name_i = 'output_'+str(o)
    output_i = layers.Dense(nb_letters, activation='softmax', dtype='float32', name=name_i)(x)
    outputs.append(output_i)
    losses[name_i] = 'categorical_crossentropy'

model = keras.models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
              loss=losses,
              metrics=['accuracy'])       

In [6]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
hl_1 (Dense)                    (None, 2048)         10240       main_input[0][0]                 
__________________________________________________________________________________________________
output_0 (Dense)                (None, 10)           20490       hl_1[0][0]                       
__________________________________________________________________________________________________
output_1 (Dense)                (None, 10)           20490       hl_1[0][0]                       
__________________________________________________________________________________________________
output_2 (

In [7]:
def display_x(x):
    words = []
    for w in x:
        word = ''
        for c in w:
            word += chr(c)
        words.append(word)
   
    print(words)

In [8]:
# ALL IN ONE STEP ('OSSIFIED')

x = create_inputs(nb_words, nb_chars, nb_letters)
display_x(x[:3])
print('x:\n', x[:3], 'out of ',len(x))
print()

# process the x data as useful ANN input data
scaler = StandardScaler()
x_train  = scaler.fit_transform(x)

print('x_train:\n', x_train[:3], 'out of ',len(x_train))
print()

# create output data for training
y = encrypt(x, nb_words, nb_chars)
print('y:\n', y)
print()
# process the y data as useful ANN output data
y_train = keras.utils.to_categorical(y, nb_letters)
print('y_train:\n', y_train[:3], 'out of ',len(y_train))

['cife', 'bdaj', 'hgai']
x:
 [[ 99 105 102 101]
 [ 98 100  97 106]
 [104 103  97 105]] out of  3000

x_train:
 [[-0.83847761  1.22588888  0.17948421 -0.21828492]
 [-1.18291591 -0.53510836 -1.55970385  1.51688934]
 [ 0.88371384  0.52148998 -1.55970385  1.16985449]] out of  3000

y:
 [[2 8 5 4]
 [1 3 0 9]
 [7 6 0 8]
 ...
 [3 4 5 0]
 [7 8 6 5]
 [1 6 1 0]]

y_train:
 [[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]] out of  3000


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
def get_sub_sub_set(y, c_ref, nb_chars):
    '''Retrieve the probalities of the i-th char'''
    nb_samples = len(y)
    
    yi = np.zeros((nb_samples, nb_letters), dtype=int)
    
    for s in range(nb_samples):
        for c in range(nb_chars):
            #print('ysl:',y[s][0][l_i])
            if c == c_ref:
                yi[s] = y[s, c]
                
    return yi

In [10]:
y_train2 = []
for c in range(nb_chars):
    yi_train = get_sub_sub_set(y_train, c, nb_chars)
    y_train2.append(yi_train)

In [11]:
y_train2[:3]

[array([[0, 0, 1, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]])]

In [12]:
history = model.fit(x_train, y_train2, epochs=10, batch_size=1,  verbose=1)

Epoch 1/10
3000/3000 [==============================] - 5s 2ms/step - loss: 4.9601 - output_0_loss: 1.2525 - output_1_loss: 1.2303 - output_2_loss: 1.2373 - output_3_loss: 1.2399 - output_0_acc: 0.4817 - output_1_acc: 0.5027 - output_2_acc: 0.4880 - output_3_acc: 0.5000
Epoch 2/10
3000/3000 [==============================] - 4s 1ms/step - loss: 2.5073 - output_0_loss: 0.6481 - output_1_loss: 0.6301 - output_2_loss: 0.6027 - output_3_loss: 0.6264 - output_0_acc: 0.7530 - output_1_acc: 0.7677 - output_2_acc: 0.7753 - output_3_acc: 0.7717
Epoch 3/10
3000/3000 [==============================] - 4s 1ms/step - loss: 1.6096 - output_0_loss: 0.4365 - output_1_loss: 0.4013 - output_2_loss: 0.3854 - output_3_loss: 0.3864 - output_0_acc: 0.8530 - output_1_acc: 0.8660 - output_2_acc: 0.8703 - output_3_acc: 0.8790
Epoch 4/10
3000/3000 [==============================] - 4s 1ms/step - loss: 1.1461 - output_0_loss: 0.2886 - output_1_loss: 0.2800 - output_2_loss: 0.2964 - output_3_loss: 0.2810 - output

In [13]:
def print_readable_prediction(prediction, nb_words):
    
    words = [''] * nb_words
    
    c_i = 0
    for char_prediction in prediction:

        s_i = 0
        for sample_char_prediction in char_prediction:

            i_letter = 0
            best_value = -float('inf')
            best_letter = -1
            for letter_prediction in sample_char_prediction:
                if letter_prediction > best_value:
                    best_value = letter_prediction
                    best_letter = i_letter
                i_letter += 1
            words[s_i] += str(best_letter)
            words[s_i] += '-'
            s_i += 1
        c_i += 1
    print(words)

In [14]:
nb_words = 2

x_test = create_inputs(nb_words, nb_chars, nb_letters)
display_x(x_test)
print("x_test=\n", x_test)

x_test_n  = scaler.transform(x_test)
print("x_test_mean=\n", x_test_n)
print('-->')

prediction = model.predict(x_test_n)
#print(prediction)
print('prediction')
print_readable_prediction(prediction, nb_words)

print('check prediction')
y_test = encrypt(x_test, nb_words, nb_chars)
print("y_test=\n", y_test)

['jahi', 'fhad']
x_test=
 [[106  97 104 105]
 [102 104  97 100]]
x_test_mean=
 [[ 1.57259043 -1.59170671  0.87515943  1.16985449]
 [ 0.19483726  0.87368943 -1.55970385 -0.56531977]]
-->
prediction
['9-0-8-9-', '5-7-0-3-']
check prediction
y_test=
 [[9 0 7 8]
 [5 7 0 3]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
